In [40]:
model_name = "t5-small"

# Libraries installation

Install Transformers libaries, framework seqeval, some extensions: unidecode for formatting accent language, datasets for creating a Dataset,...

In [3]:
# #Install required libraries
# !pip install datasets transformers evaluate seqeval unidecode google.colab
# !pip install transformers[torch]

## WandB

In [4]:
# !pip install wandb
# !wandb login

In [5]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/text-classification/run_glue.py

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
# !pip install -q git+https://github.com/huggingface/transformers

In [7]:
# import wandb

In [8]:
# wandb.login()

In [9]:
# %env WANDB_LOG_MODEL=true

## Import required libraries

In [10]:
#Import libraries to project
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
import torch
from datasets import Dataset
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import json
import pandas as pd
from unidecode import unidecode
import os
import logging

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for unidecode from https://files.pythonhosted.org/packages/e4/63/7685ef40c65aba621ccd2524a24181bf11f0535ab1fdba47e40738eacff6/Unidecode-1.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow>=8.0.0 from https://files.pythonhosted.org/packages/d4/f0/607f50ec87ac4775d6124855ae6be2c48bab58aa0a660ccd46e9af52bcd9/pyarrow-14.0.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotf


[notice] A new release of pip is available: 23.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Mount Google Drive to Google Colab

In [12]:
# #Mount Drive to Colab
# from google.colab import drive
# drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

# Data preparation

Read training set from dataset stored in Google Drive.
Create a list of input and output sequences

In [14]:
#Read datasets

root_data_dir = "../Datasets/NER/BiodivNER"

dataset = "train"
train_csv_file_path = "train.csv"
val_csv_file_path = "dev.csv"
test_csv_file_path = "test.csv"

In [69]:
def loadData(csv_file_path):
  dataset_path = os.path.join(root_data_dir, csv_file_path)
  data = pd.read_csv(dataset_path, encoding="utf-8")
  data = data.fillna(method="ffill")
  return data

In [70]:
data = loadData(train_csv_file_path)
val_data = loadData(val_csv_file_path)
test_data = loadData(test_csv_file_path)

In [71]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                        s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [72]:
getter = SentenceGetter(data)
sentences = getter.sentences
sent = getter.get_next()
print(sent)

[('bottom', 'O'), ('board', 'O'), ('debris', 'O'), (',', 'O'), ('frames', 'O'), (',', 'O'), ('landing', 'O'), ('platforms', 'O'), (')', 'O'), (',', 'O'), ('and', 'O'), ('isolates', 'O'), ('of', 'O'), ('microbes', 'O'), (',', 'O'), ('parasites', 'O'), ('and', 'O'), ('pathogens', 'O'), ('from', 'O'), ('honey', 'B-Organism'), ('bees', 'I-Organism'), ('.', 'O')]


In [73]:
getter_val = SentenceGetter(val_data)
sentences_val = getter_val.sentences
sent_val = getter_val.get_next()
print(sent_val)

[('However', 'O'), (',', 'O'), ('the', 'O'), ('lack', 'O'), ('of', 'O'), ('correlation', 'O'), ('between', 'O'), ('dung', 'O'), ('beetle', 'O'), ('community', 'B-Environment'), ('characteristics', 'O'), ('and', 'O'), ('dung', 'O'), ('removal', 'O'), ('highlights', 'O'), ('the', 'O'), ('need', 'O'), ('for', 'O'), ('further', 'O'), ('research', 'O'), ('into', 'O'), ('spatial', 'O'), ('variation', 'O'), ('in', 'O'), ('biodiversityÃ¢â\x82¬â\x80\x9cecosystem', 'O'), ('function', 'O'), ('relationships', 'O'), ('and', 'O'), ('how', 'O'), ('the', 'O'), ('results', 'O'), ('of', 'O'), ('such', 'O'), ('studies', 'O'), ('are', 'O'), ('affected', 'O'), ('by', 'O'), ('methodological', 'O'), ('choices', 'O'), ('.', 'O')]


In [74]:
getter_test = SentenceGetter(test_data)
sentences_test = getter_test.sentences
sent_test = getter_test.get_next()
print(sent_test)


[('The', 'O'), ('primacy', 'O'), ('of', 'O'), ('either', 'O'), ('species', 'B-Quality'), ('or', 'O'), ('functional', 'O'), ('group', 'O'), ('richness', 'B-Quality'), ('effects', 'O'), ('depended', 'O'), ('on', 'O'), ('the', 'O'), ('sequence', 'O'), ('of', 'O'), ('testing', 'O'), ('these', 'O'), ('terms', 'O'), (',', 'O'), ('indicating', 'O'), ('that', 'O'), ('both', 'O'), ('aspects', 'O'), ('of', 'O'), ('richness', 'B-Quality'), ('were', 'O'), ('congruent', 'O'), ('and', 'O'), ('complementary', 'O'), ('to', 'O'), ('expected', 'O'), ('strong', 'O'), ('effects', 'O'), ('of', 'O'), ('legume', 'O'), ('presence', 'O'), ('and', 'O'), ('grass', 'B-Organism'), ('presence', 'O'), ('on', 'O'), ('plant', 'B-Organism'), ('chemical', 'B-Quality'), ('composition', 'I-Quality'), ('.', 'O')]


In [75]:
def get_text_tags_lists(sentences):
  texts = []
  tags = []
  for sent in sentences: #list of tuples
    sent_texts = []
    sent_tags = []
    for tuple1 in sent:
      sent_texts.append(tuple1[0])
      sent_tags.append(tuple1[1])

    texts.append(sent_texts)
    tags.append(sent_tags)
  return texts, tags

In [76]:
train_texts, train_tags = get_text_tags_lists(sentences)
val_texts, val_tags = get_text_tags_lists(sentences_val)
test_texts, test_tags = get_text_tags_lists(sentences_test)

In [23]:
#Create a list of texts
def create_input_seq(texts):
  inputs = texts
  input_sequences = []

  for sentence in inputs:
    input_sequences.append(' '.join(sentence))

  return input_sequences

In [124]:
train_inputs_raw = create_input_seq(train_texts)
val_inputs_raw = create_input_seq(val_texts)
test_inputs_raw = create_input_seq(test_texts)

Convert problem into text-to-text format by formatting an output which is a sequence of class:entity pairs separated by a semicolon ';'

In [25]:
def format_targets(text, label):
  s = ""
  texts = []
  for i in range(len(text)-1):
    if label[i] != 'O':
      s += text[i] + " "
      if label[i+1] == 'O' or label[i+1][0:1] == 'B':
        s = label[i][2:] + ": " + s
        texts.append(s.strip())
        s = ""
  texts = "; ".join(texts)
  return texts

In [26]:
str1 = format_targets(train_texts[0], train_tags[0])
str1

'Quality: Density; Phenomena: Treatment; Quality: Dead; Environment: Garden; Organism: Seedling; Phenomena: growth; Quality: biomass; Environment: plant communities; Quality: species richness; Environment: tropical forest ecosystems'

 Convert them back to sentence format

In [27]:
def create_output_seq(texts, tags):
  output_sequences = []

  for i in range(len(texts)):
    text = texts[i]
    label = tags[i]

    target = format_targets(text, label)
    output_sequences.append(target)

  return output_sequences

In [126]:
train_outputs_raw = create_output_seq(train_texts, train_tags)
val_outputs_raw = create_output_seq(val_texts, val_tags)
test_outputs_raw = create_output_seq(test_texts, test_tags)

In [208]:
def has_non_ascii(sentence):
    return any(ord(char) > 127 for char in sentence)

def clean_non_ascii(inputs, labels):
    clean_inputs = []
    clean_labels = []

    for sentence, label in zip(inputs, labels):
        if has_non_ascii(sentence):
            continue
        
        clean_inputs.append(sentence)
        clean_labels.append(label)

    return clean_inputs, clean_labels

def clean_non_ascii_test(texts, tags):
    clean_inputs = []
    clean_labels = []

    for sentence, label in zip(texts, tags):
        sentence2 = ' '.join(sentence)
        if has_non_ascii(sentence2):
            continue
        
        clean_inputs.append(sentence)
        clean_labels.append(label)

    return clean_inputs, clean_labels

In [209]:
train_inputs, train_outputs = clean_non_ascii(train_inputs_raw, train_outputs_raw)
val_inputs, val_outputs = clean_non_ascii(val_inputs_raw, val_outputs_raw)
test_texts_c, test_tags_c = clean_non_ascii_test(test_texts, test_tags)

In [210]:
print(test_texts_c)

[['Combining', 'metabolic', 'and', 'food-', 'web', 'theory', ',', 'we', 'calculate', 'annual', 'energy', 'fluxes', 'to', 'model', 'impacts', 'of', 'land-use', 'intensification', 'on', 'multitrophic', 'ecosystem', 'functioning', '.'], ['The', 'primacy', 'of', 'either', 'species', 'or', 'functional', 'group', 'richness', 'effects', 'depended', 'on', 'the', 'sequence', 'of', 'testing', 'these', 'terms', ',', 'indicating', 'that', 'both', 'aspects', 'of', 'richness', 'were', 'congruent', 'and', 'complementary', 'to', 'expected', 'strong', 'effects', 'of', 'legume', 'presence', 'and', 'grass', 'presence', 'on', 'plant', 'chemical', 'composition', '.'], ['Full', 'ant', 'species', 'name', '.'], ['If', 'it', 'would', 'not', 'be', 'considered', 'greater', 'than', '10', ',', 'it', 'would', 'not', 'be', 'considered', 'in', 'the', 'outer', 'subplots.Helper', ';', 'Datagroup', 'description', ':', 'Helper', ')', 'dimensionless', 'real', 'Helper', 'Helper', 'column', 'to', 'understand', 'other', 'col

In [150]:
for row in train_outputs[:10]:
  print(row, "\n")

Quality: Density; Phenomena: Treatment; Quality: Dead; Environment: Garden; Organism: Seedling; Phenomena: growth; Quality: biomass; Environment: plant communities; Quality: species richness; Environment: tropical forest ecosystems 

Organism: honey bees 

Environment: soil 

Environment: riparian reserves; Environment: forest; Environment: riparian reserves; Matter: oil palm 

Organism: species 

Phenomena: precipitation; Quality: rainfall amount; Environment: field; Phenomena: rainfall; Quality: rainfall amount; Environment: field; Quality: average rainfall intensity; Environment: field; Quality: peak rainfall intensity; Environment: field; Quality: intensities; Environment: field; Quality: rainfall amount; Quality: average rainfall intensity; Quality: peak rainfall intensity; Quality: intensities; Environment: field; Quality: intensities; Phenomena: Precipitation; Phenomena: Precipitation 

Environment: forest; Environment: tropical forest landscape; Quality: area; Environment: cano

In [154]:
train_inputs[11], train_outputs[11]

('-- - 6 digit metal tags starting with 3 were also used for woody debris items -- the reference file linking metal tags to tree individual tags is called `` Tree stem reference list for the Comparative Study Sites ( CSPs ) ) ( neigh.label : label ID of neighbouring individual : We locate every tagged individual CWD piece .',
 'Matter: woody debris items')

# Training preparation

We first load a ViT5 tokenizer and a pretrained model powered by Transformers library

In [184]:
#Create a tokenizer and a model supported by ViT5
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
# model.to("cuda")

Define a function which converts data to tokens

In [185]:
#Function for tokenizing texts and labels
def preprocess(examples):
  model_inputs = tokenizer(
      examples["inputs"], max_length=512, truncation=True, padding=True
  )
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(
        examples["labels"], max_length=512, truncation=True, padding=True
    )
  model_inputs["labels"] = labels['input_ids']
  model_inputs['input_ids'] = model_inputs['input_ids']
  return model_inputs

Define a dictionary that holds inputs and their corresponding labels

Create a dataset from that dictionary and apply the tokenization process to every data sample with map function

In [186]:
#Tokenise dataset
def dict_to_dataset(inputs, outputs):
  dict_obj = {'inputs':inputs, 'labels':outputs}
  dataset = Dataset.from_dict(dict_obj)
  tokenized_datasets = dataset.map(preprocess, batched=True, num_proc=8)

  return tokenized_datasets

In [187]:
train_tokenized = dict_to_dataset(train_inputs, train_outputs)
val_tokenized = dict_to_dataset(val_inputs, val_outputs)
test_tokenized = dict_to_dataset(test_inputs, test_outputs)

Map (num_proc=8):   0%|          | 0/1776 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

In [33]:
logging_dir="./log"
logging_dir_name="training_logs"
logging_dir_path = os.path.join(logging_dir, logging_dir_name)
os.makedirs(logging_dir_path, exist_ok=True)

In [34]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [35]:
from seqeval import metrics as seqeval_metrics

In [36]:
def compute_metrics(p):
  predictions, labels = p.predictions, p.label_ids
  flat_predictions = [label for sentence_labels in predictions for label in sentence_labels]
  flat_labels = [label for sentence_labels in labels for label in sentence_labels]
  accuracy = seqeval_metrics.accuracy_score([flat_labels], [flat_predictions])
  return {"accuracy": accuracy}

Set training arguments for training process

In [37]:
#Arguments for training model
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")
training_args = Seq2SeqTrainingArguments('/content/gdrive/Shareddrives/CS 198 NLP for Biodiversity (Plants)/Model Data',
                                      report_to = 'wandb',
                                      do_train=True,
                                      do_eval=True,
                                      num_train_epochs=5,
                                      learning_rate=2e-5,
                                      warmup_ratio=0.00,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,

                                      logging_steps=50,
                                      group_by_length=True,
                                      # save_strategy="epoch",
                                      save_total_limit=0,
                                      #eval_steps=1,
                                      evaluation_strategy="epoch",
                                      # evaluation_strategy="no",
                                      # fp16=True,
                                      # eval_accumulation_steps=50
                                      )

Make sure to push the model to GPU for optimisation

In [42]:
model.to("cuda" if torch.cuda.is_available() else "cpu")

AssertionError: Torch not compiled with CUDA enabled

# Train Named Entity Recognition model

In [ ]:
# # #Procession of training model
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_tokenized,
#     eval_dataset=val_tokenized,
#     data_collator=data_collator,
# )

# trainer.train()

Epoch,Training Loss,Validation Loss
1,0.065700,0.156220
2,0.075600,0.144565
3,0.043100,0.135807
4,0.049900,0.132062
5,0.071300,0.130788


Checkpoint destination directory /content/gdrive/Shareddrives/CS 198 NLP for Biodiversity (Plants)/Model Data/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory /content/gdrive/Shareddrives/CS 198 NLP for Biodiversity (Plants)/Model Data/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=2400, training_loss=0.05676367496450742, metrics={'train_runtime': 873.5687, 'train_samples_per_second': 10.978, 'train_steps_per_second': 2.747, 'total_flos': 1297927876116480.0, 'train_loss': 0.05676367496450742, 'epoch': 5.0})

In [ ]:
trainer.evaluate(eval_dataset=train_tokenized)

KeyboardInterrupt: ignored

In [39]:
# Assuming that 'model' is your Seq2Seq model
# model.save_pretrained()

# Inference

Load fine-tuned model stored in Google Drive with a Tokenizer by ViT5

In [188]:
#Load model saved as a checkpoint
from transformers import AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained('t5-small')
model = AutoModelForSeq2SeqLM.from_pretrained("../Models/t5-small_001/")
# model = trainer.model
# model.to("cuda")

In [189]:
#Total number of trainable parameters
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

60506624

In [190]:
model.to('cpu')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

To use the model at its best, make sure the data fed to the model is word-level,
 if it's not, simply use an RDR-Segmenter!

The cell below helps produce target text with 'generate' method

In [191]:
sentence = train_inputs[100]
# sentence = eval_input_sequences[23]
encoding = tokenizer(sentence, return_tensors="pt", max_length=512)
input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]
input_ids

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


tensor([[ 1636,    37, 21965,  7322,    19,   787,    57,   309,  1713,  3316,
           448,  2990,    55,  4505,     3,   102,   834,    23,     2,   357,
             3,     6,    28,     3,   102,   834,    23,  9085,     8,  5237,
         15025,    13,     8,    34,   107,  3244,     3,     5,     1]])

In [192]:
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=512,
)

In [193]:
print(outputs)

tensor([[    0,  6495,    10, 21965,  7322,   117,  6495,    10, 15025,     1]])


In [194]:
labels = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
print("Results: ", labels)
print("Actual labels:", train_outputs[50])

Results:  Quality: Simpson diversity; Quality: abundance
Actual labels: Organism: plant; Quality: herbivore diversity; Phenomena: biodiversity loss; Quality: trophic levels


In [195]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

'word_labels' takes into account two parameters 'sentence' and 'labels' to calculate the correct BIO label and map them to a list

In [196]:
def word_labels(sentence, labels):
  predictions = ["O" for i in range(len(sentence.split()))]
  if labels != '':
    list_labels = labels.split(";")
    sent = sentence.split()

    start = 0
    for i in range(len(list_labels)):
      sub_list = list_labels[i].split(":")
      # print(list_labels[i])
      # print(sub_list)
      class_entity = sub_list[0].strip() # location, organization, age,...
      named_entity = sub_list[1].strip().lower() # Ha Noi, London, 433,...
      named_entity_element = named_entity.split() # Ha, Noi, London, 433, Soc Trang,...

      flist = []
      for i in range(len(named_entity_element)):
        if named_entity_element[i][-1] == ',':
          entity1 = named_entity_element[i][0:len(named_entity_element[i])-1]
          entity2 = ","
          flist.append(entity1)
          flist.append(entity2)
        else:
          flist.append(named_entity_element[i])

      named_entity_element = flist
      for i in range(len(named_entity_element)):
        try:
          findex = sent.index(named_entity_element[i], start)
          start = findex + 1
          f_class = ""
          if i == 0:
            f_class = "B-" + class_entity
          else:
            f_class = "I-" + class_entity
          predictions[findex] = f_class
        except:
          pass

  return predictions

# reverse the subword tokens to their real words

Load dataset for devaluation

In [168]:
# #Read test set
# evalWord = [json.loads(line) for line in open('gdrive/MyDrive/Ct550/word/test_word.json', 'r', encoding='utf-8')]
# evalWordData = pd.DataFrame(evalWord)
# evalWordData = evalWordData.rename(columns={'tags':'target_text', 'words':'source_text'})

Create lists of input and output sequences

In [211]:
# #Create a list of texts
eval_input_data = test_texts_c
test_inputs = []

for sentence in eval_input_data:
  test_inputs.append(" ".join(sentence))

# #Create a list of targets
eval_output_data = test_tags_c
test_outputs = []

for sentence in eval_output_data:
  test_outputs.append(" ".join(sentence))

In [212]:
# eval_input_sequences
test_inputs[8]

'Or the allometries are not used to extrapolate the biomass of the first item ( for example a tree dbh ) . )'

In [213]:
model.to("cuda" if torch.cuda.is_available() else "cpu")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

The process below predicts targets for 2k data samples in the dev set

In [214]:
predictions = []
references = []

In [215]:
print(test_inputs[47])

Chlamydia caviae infection alters abundance but not composition of the guinea pig vaginal microbiota In humans , the vaginal microbiota is thought to be the first line of defense again pathogens including Chlamydia trachomatis .


In [216]:
print(predictions[8])

IndexError: list index out of range

In [222]:
# for i in test_inputs:
for i in range(len(test_inputs)):
  print (i)
  sentence = test_inputs[i]
  tokenized_input = tokenizer(sentence, return_tensors='pt', max_length=512)
  input_ids = tokenized_input['input_ids'].to("cuda" if torch.cuda.is_available() else "cpu")
  attention_mask = tokenized_input['attention_mask'].to("cuda" if torch.cuda.is_available() else "cpu")

  try:
        outputs = model.generate(
            input_ids=input_ids, attention_mask=attention_mask, max_length=512
        )
        # print(outputs[0])
        labels = tokenizer.decode(outputs[0], clean_up_tokenization_spaces=True, skip_special_tokens=True)
        # print("prev sentence:", sentence)
        # print("prev_label:", labels)
        sentence = unidecode(sentence).lower()
        labels = unidecode(labels)
        # print("decoded_sentence:", sentence)
        # print("decoded_label:", labels)
        flabels = word_labels(sentence, labels)
        if len(flabels) != len(test_outputs[i].split()):
          print(f"Unequal prediction and reference!")
          continue
        predictions.append(flabels)
        references.append(test_outputs[i].split())

  except Exception as e:
      print(f"Error in iteration {i}: {str(e)}")

0
1
2
3
4
5
6
7
Error in iteration 7: list index out of range
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
Error in iteration 33: list index out of range
34
35
36
37
38
39
Error in iteration 39: list index out of range
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
Error in iteration 104: list index out of range
105
106
107
108
109
110
111
112
113
114
115
116
117
Error in iteration 117: list index out of range
118
119
120
121
122
123
Error in iteration 123: list index out of range
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
Error in iteration 141: list index out of range
142
143
144
145
146
147
148
149
150
Error in iteration 150: list index out of range
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
Error in iteration

In [99]:
# List index out of range error
print(test_inputs[8])
print(test_inputs[36])
print(test_inputs[43])

Number of individuals Organism count ( Hemiptera_sum ) , Hemiptera_sum Counting the individuals of a given taxon , also called abundance .
Number of individuals Organism count ( Tetramorium.wroughtonii ) , Tetramorium.wroughtonii Counting the individuals of a given taxon , also called abundance .
-- d ) Compare species-specific sensitivities to snow break as a trait i ) Calculate the likelihood of being subject to snowbreak given the species , size and potentially the relative position using logistic regression ii ) Calculate the likelihood of snag survival ; this requires re-sampling iii ) Estimate the down-slope transport of deadwood pieces iv ) Estimate the time it takes for deadwood to collapse and touch the forest floor -- -- - File asset Plot_skizze.pdf : plot sketch how the debris objects are selected - File asset sp10 CWD_protocol_Gutianshan_AF_20081215_kn.doc : Inventory protocol , should be included into methods -- -- - 2 ) Objective allometries : use some of the downed or al

In [110]:
print(test_outputs[8])
print(test_outputs[36])
print(test_outputs[43])

O O O B-Quality I-Quality O O O O O O O O O O O B-Organism O O O B-Quality O
O O O B-Quality I-Quality O O O O O O O O O O O B-Organism O O O B-Quality O
O O O O O O O B-Phenomena I-Phenomena O O B-Quality O O O O O O O O O O O O B-Organism O B-Quality O O O B-Quality I-Quality O O O O O O O O O B-Phenomena I-Phenomena O O O O O O O O O O O B-Matter O O O O O O O O O B-Matter O O O O O B-Environment I-Environment O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-Organism I-Organism O O O O O O B-Quality O O O O O B-Quality I-Quality B-Matter I-Matter I-Matter O O O O B-Phenomena I-Phenomena I-Phenomena B-Organism B-Phenomena I-Phenomena B-Quality B-Quality O O O O O B-Quality O B-Quality B-Environment B-Organism O O B-Phenomena B-Matter I-Matter O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-Organism O O O O O O O O O O O O O B-Quality O O O O O O B-Quality O O O O O O O O O O O O O O O O O O O O B-Quality O O O O O B-Organism O O O O 

In [115]:
# List index out of range error
for row in predictions:
    print(row)

In [97]:
for row in references:
    print(row)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Phenomena', 'I-Phenomena', 'O', 'O', 'B-Environment', 'O', 'O']
['O', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Organism', 'O', 'O', 'B-Organism', 'B-Quality', 'I-Quality', 'O']
['B-Quality', 'I-Quality', 'I-Quality', 'I-Quality', 'O']


In [178]:
# Unequal prediction and reference error
print(test_inputs[23])
print(test_inputs[47])
print(test_inputs[81])
print(test_inputs[83])

Acronym : SLA Specific leaf area ; Datagroup description : Specific leaf area ; Datagroup description : Specific leaf area Leaf dry matter content ( LDMC ) , mg/g LDMC Leaf dry matter content , LDMC ( LDMC : Leaf dry matter content ) dimensionless real Leaf dry matter content Leaf dry matter content , LDMC Leaf dry matter content Nitrogen ( N ) , mg/g N Nitrogen concentration ( N : Nitrogen content ; Datagroup description : Nitrogen ; Datagroup description : Nitrogen ) dimensionless real Nitrogen Nitrogen concentration Nitrogen content ; Datagroup description : Nitrogen ; Datagroup description : Nitrogen Elemental Analyzer Carbon ( C ) , mg/g C measurements of Carbon such as concentrations , pools , amount ( C : Carbon content ; Instrumentation : Elemental Analyzer ) dimensionless real Carbon measurements of Carbon such as concentrations , pools , amount Carbon content ; Instrumentation : Elemental Analyzer Carbon Nitrogen ratio ( CN ) , g/g CN Carbon Nitrogen ratio ( CN : Carbon to Ni

In [ ]:
for row in references[:10]:
  print(row)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Phenomena', 'I-Phenomena', 'O', 'O', 'B-Environment', 'O', 'O']
['O', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Organism', 'O', 'O', 'B-Organism', 'B-Quality', 'I-Quality', 'O']
['B-Quality', 'I-Quality', 'I-Quality', 'I-Quality', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'B-Organism', 'I-Organism', 'O', 'O', 'B-Organism', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [ ]:
for row in predictions[:10]:
  print(row)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'I-Quality', 'O', 'O', 'O', 'O', 'B-Quality', 'I-Quality', 'O', 'O', 'B-Quality', 'I-Quality', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'I-Quality', 'I-Quality', 'O']
['O', 'O', 'B-Quality', 'I-Quality', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'B-Organism', 'I-Organism', 'O', 'O', 'B-Organism', 'I-Organism', 'B-Organism', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

# resultssss

Use seqeval framework introduced to calculate scores F1/precision/recall for every class entity and overall score for model performance

In [223]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

precision = precision_score(references, predictions)
recall = recall_score(references, predictions)
f1 = f1_score(references, predictions)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')

# You can also print a detailed classification report
print(classification_report(references, predictions))

# result = seqeval.compute(predictions=predictions, references=references)
# resultb

Precision: 0.2157
Recall: 0.1990
F1-Score: 0.2070
                   precision    recall  f1-score   support

<pad> Environment       0.00      0.00      0.00         0
   <pad> Organism       0.00      0.00      0.00         0
    <pad> Quality       0.00      0.00      0.00         0
      Environment       0.16      0.04      0.07       268
         Location       0.00      0.00      0.00        56
           Matter       0.00      0.00      0.00       102
         Organism       0.36      0.26      0.30       442
        Phenomena       0.00      0.00      0.00       109
          Quality       0.22      0.35      0.27       455

        micro avg       0.22      0.20      0.21      1432
        macro avg       0.08      0.07      0.07      1432
     weighted avg       0.21      0.20      0.19      1432



c:\Users\shann\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\shann\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [221]:
print(len(references))

209
